In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from math import ceil
import psycopg2 as pg
import psycopg2.extras
import pandas.io.sql as sqlio

import db_connection as dbc

# Dimension _'Date'_

This notebooks describes the ETL process to fed the 'Date' dimension table.

## Introduction

The data dimension is essential in almost all data warehousing system, in the sense that all the facts could be observed in a data-time perspective.

In our case study we will only use dates and not times.

### Schema

Our dimension _'Date'_ has the following schema:

<img src="images/date_schema.jpeg" align="center"/>

During this project phase, and in order to increase the quality of the date dimension, the following attributes were added:

* weekday;
* quarter;
* semester;
* holiday

### Granularity

Each record in the date dimension corresponds to a day.

### Hierarchies

Some examples of hierarchies that could be identified:

* year -> semester -> quarter -> month -> week -> day;
* year -> season -> month -> week -> weekday / weekend -> day;
* year -> season -> quarter -> month -> weekday -> weekend

## Exploratory Data Analysis (EDA)

In this section we will do a small EDA in order to get a sense of the input data of the ETL process.

### Read the data

Read listings_al.csv, which has the dates needed for the listings fact table.

In [2]:
# read listings_al.csv data file
listings_al_file_path = '../data/listings_al.csv'
df_listings = pd.read_csv(listings_al_file_path,sep=',')

C:\Users\joao_\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Read calendar.csv file, which has the dates needed for the booking fact table.

In [3]:
# read calendar.csv data file
calendar_file_path = '../data/airbnb/calendar.csv'
df_calendar = pd.read_csv(calendar_file_path,sep=',')

Read csv file that contains information about Portugal national holidays.

In [4]:
holiday_file_path = '../data/feriados_nacionais.csv'
df_holidays = pd.read_csv(holiday_file_path)
df_holidays.head()

,Title,Given planned earliest start,Given planned earliest end,Notes,Assigned Resources,Additional Title
0,Ano Novo,1/1/2020,1/1/2020,feriado nacional em Portugal | feriados.com.pt...,NaN,NaN
1,Carnaval,2/24/2020,2/24/2020,feriado regional | feriados.com.pt. All data w...,NaN,NaN
2,Sexta-Feira Santa,4/10/2020,4/10/2020,feriado nacional em Portugal | feriados.com.pt...,NaN,NaN
3,Páscoa,4/12/2020,4/12/2020,feriado nacional em Portugal | feriados.com.pt...,NaN,NaN
4,Segunda-feira de Páscoa,4/13/2020,4/13/2020,feriado regional | feriados.com.pt. All data w...,NaN,NaN


### Convert dates from string 

In [5]:
len(np.unique(df_listings['host_id'].values))

6976

In [6]:
# convert string to date
df_listings['DataRegisto'] = [datetime.strptime(d.split('.')[0], "%Y-%m-%dT%H:%M:%S") for d in df_listings['DataRegisto']] # split into YYYY-MM-DD HH:MM:SS
df_listings.shape

(17168, 147)

In [ ]:
df_calendar['date'] = [datetime.strptime(d, "%Y-%m-%d") for d in df_calendar['date']] # split into YYYY-MM-DD
df_calendar.shape

In [ ]:
df_calendar.head() 

In _calendar.csv_, there are approximately 365 records for each listing. Each record indicates if the listing is available in that day.

Now lets drop the duplicates from df_calendar.

In [ ]:
# remove df_calendar duplicates and save it new dataframe
df_calendar2 = df_calendar.drop_duplicates(subset=['date'])
df_calendar2.shape

Read and process holidays dates.

In [ ]:
holidays_dates = df_holidays.iloc[:,1].values[:-1]
holidays_dates = [datetime.strptime(d, "%m/%d/%Y") for d in holidays_dates]

### Exploratory data analysis

We do a quick exploration of the data available in _listings_al.csv_.

In [ ]:
# count by year
years = np.array([d.year for d in df_listings['DataRegisto']]) # makes list of year in each listing's date
df_years = pd.DataFrame(years,columns=['year']) # convert to dataframe
df_years_graph = df_years.year.value_counts().to_frame('count').reset_index().rename(columns={'index': 'year'}).sort_values('year',ascending = True) # count each year, sort by asc.

# remove count of 2020 because it is incomplete (data exported from inside airbnb in january)
df_years_graph = df_years_graph[df_years_graph['year']!=2020]

# create column with the cumulative sum
df_years_graph['cum_count'] = df_years_graph['count'].cumsum()

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(df_years_graph['year'],df_years_graph['count'], label = 'New listings')
plt.plot(df_years_graph['year'],df_years_graph['cum_count'], label = 'Cumulative new listings')
plt.axvline(x = 2008, color = 'red')
plt.xlabel('Year')
plt.ylabel('Number of new listings')
plt.title('Number of Airbnb listings in Lisbon (all)')
plt.legend()

It seems a negligible part of the properties were registered before 2010 for lodging. This already gives us some insights: there was a clear boom in property registration with the intention to host tourists after Airbnb was created in 2008 (red line). Data from 2020 was neglected as it is incomplete (the dataset was scraped in January).

Let's plot the data just from 2008 forward.

In [ ]:
df_years_graph2010 = df_years_graph[df_years_graph['year']>= 2008]
plt.figure(figsize=(15,7))
plt.plot(df_years_graph2010['year'],df_years_graph2010['count'], label = 'New listings')
plt.plot(df_years_graph2010['year'],df_years_graph2010['cum_count'], label = 'Cumulative new listings')
plt.axvline(x = 2008, color = 'red')
plt.xlabel('Year')
plt.ylabel('Number of new listings')
plt.title('Number of Airbnb listings in Lisbon (2010 - 2019)')
plt.legend()

We can see that since 2013, the number of new listings in airbnb for Lisbon increased until 2018, from less than 500 new listings per year to more than 5000 new listings per year, which is a 10x increase. In 2019 the growth in new listings decreased compared to the previous year.

In [ ]:
df_years_graph2010

In 2010 there were 151 listings in Airbnb located in Lisbon, and in 2019 there was a total of 17155 listings, which is an increase of about 113 times.

## ETL Process

In this sections all the preprocessing will be done in order to get the data cleaned to fed the date dimension table.

The **pipeline of the ETL process** for the date dimension can be seen in the image above.

<img src="images/Pipeline_date.png" align="center"/>

The process receives as inputs 3 csv files:

* **'listings_al.csv'**: which contains the dates needed to be inserted in order the fed the listing fact table;
* **'calendar.csv'**: contains dates about weather a specific listings is available in the future or not;
* **'feriados_nacionais.csv'**: contains all the national holidays of Portugal. This data will be used to fed the 'holiday' columns in this dimension, in order to increase the quality of the analysis that could be done further.

This inputs are then transformed and processed during this notebook, in order to insert just the dates that are not in the database.

At the end of the process, a csv named 'df_listings_date.csv' is saved in the '/processed_dt/' directory. This csv contains the mapping between the date and the listings facts, and will be used in order to fed the fact tables. We use this same process in all the dimensions due to the way we distribute work among the different elements of the group.

This pipeline could be used multiple times, because it only takes into account new dates that needed to be inserted.

We start by defining functions that process each date and build its respective attributes in the dimension.

In [ ]:
def date_season(month):
    """Detects the season of the year the date belongs to"""
    if (month < 3) or (month == 12): return 'Winter'
    elif (month >= 3) and (month < 6): return 'Spring'
    elif (month >= 6) and (month < 9): return 'Summer'
    elif (month >= 9) and (month < 12): return 'Autumn'

def date_weekend(week_day):
    """Detects if date is work day or weekend"""
    if (week_day == 5) or (week_day == 6): return 'Weekend'
    else: return 'Work Day'

def week_day(d):
    """Detects weekday"""
    days = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}
    return days[d.weekday()]

def date_pk(date):
    """Builds primary key"""
    return int(date.strftime('%d%m%Y'))

def week_of_month(dt):
    """Detects the week of the month for the specified date"""
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

def getQuarter(d):
    """Detects which quarter of the year date belongs to"""
    if d.month<=3: return "Q1"
    if d.month<=6: return "Q2"
    if d.month<=9: return "Q3"
    return "Q4"

def getSemester(d):
    """Detects which semester of the year date belongs to"""
    if d.month<=6: return "S1"
    return "S2"

def is_holiday(d):
    if len(df_holidays2[(df_holidays2['Day']==d.day) & (df_holidays2['Month']==d.month)])>0: return 'Holiday'
    else: return 'Not Holiday'

Example of date dimension primary key in format 'ddMMyyyy'

In [ ]:
df_listings['DataRegisto'][0], date_pk(df_listings['DataRegisto'][0])

### Preprocessing *feriados_nacionais.csv* data

Preprocess the file in order to get a clean dataframe with the holidays.

This dataframe will be used to queries in order to know if a specific date is a national holiday or not. The method is defined above: 'def is_holiday()'.

In [ ]:
holidays_days = [d.day for d in holidays_dates]
holidays_months = [d.month for d in holidays_dates]

df_holidays2 = pd.DataFrame(np.stack((holidays_days,holidays_months), axis = 1),columns=['Day','Month'])
df_holidays2

The data in this dataframe will be used in order to fed the 'Holiday' column of the dimension.

### Preprocessing the _df_listings_ data

In this section we will process the data in order to get the attributes of the date dimension table.

In [ ]:
pks = np.array([date_pk(d) for d in df_listings['DataRegisto']])
df_listings['date_id'] = np.array([date_pk(d) for d in df_listings['DataRegisto']])
days = np.array([d.day for d in df_listings['DataRegisto']])
weeks = np.array([week_of_month(d) for d in df_listings['DataRegisto']])
week_days = np.array([week_day(d) for d in df_listings['DataRegisto']])
week_ends = np.array([date_weekend(d.weekday()) for d in df_listings['DataRegisto']])
months = np.array([d.month for d in df_listings['DataRegisto']])
seasons = np.array([date_season(d.month) for d in df_listings['DataRegisto']])
quarters = np.array([getQuarter(d) for d in df_listings['DataRegisto']])
semesters = np.array([getSemester(d) for d in df_listings['DataRegisto']])
holidays = np.array([is_holiday(d) for d in df_listings['DataRegisto']])

After processing each date in the dataframe and building a list of values for each attribute, we merge them together in a DataFrame.

In [ ]:
columns = ['date_id','day','week', 'month','year','season','weekend','weekday','quarter','semester','holiday']
df_date_dimension = pd.DataFrame(np.stack((pks,days,weeks, months,years, seasons,week_ends,week_days,quarters,semesters,holidays),axis=-1), columns = columns)
df_date_dimension['date_id'] = df_date_dimension['date_id'].astype(int) 
df_date_dimension.shape

Removal of duplicates

In [ ]:
df_date_dimension = df_date_dimension.drop_duplicates(subset=['date_id'])
df_date_dimension.shape

### Processing the _df_calendar_ data

In this section we will process the dates needed in order to feed the bookings fact table.

In [ ]:
df_calendar2.head()

Process the data.

In [ ]:
pks2 = [date_pk(d) for d in df_calendar2['date']]
days2 = [d.day for d in df_calendar2['date']]
weeks2 = [week_of_month(d) for d in df_calendar2['date']]
week_days2 = [week_day(d) for d in df_calendar2['date']]
week_ends2 = [date_weekend(d.weekday()) for d in df_calendar2['date']]
months2 = [d.month for d in df_calendar2['date']]
seasons2 = [date_season(d.month) for d in df_calendar2['date']]
quarters2 = [getQuarter(d) for d in df_calendar2['date']]
semesters2 = [getSemester(d) for d in df_calendar2['date']]
years2 = [d.year for d in df_calendar2['date']]
holidays2 = [is_holiday(d) for d in df_calendar2['date']]

Create a dataframe with dates that came from the 'calendar.csv' input file.

In [ ]:
df_calendar_processed = pd.DataFrame(np.stack((pks2,days2,weeks2,months2,years2,seasons2,week_ends2,week_days2,quarters2,semesters2,holidays2),axis=-1), columns = columns)
df_calendar_processed['date_id'] = df_calendar_processed['date_id'].astype(int)

### The ETL output

The output of the ETL process is the dataset with the data available to insert in the dataset

Lets merge the two dataframes (dates needed both for booking and listing facts), in order to get our final dataframe.

In [ ]:
df_date_dimension_insert = pd.concat([df_calendar_processed,df_date_dimension])
df_date_dimension_insert['date_id'] = df_date_dimension_insert['date_id'].astype(int)
df_date_dimension_insert

In [ ]:
df_date_dimension_insert.to_csv('../processed_dt/dimension_data.csv')

### Insert data into DB

In this section we will remove the dates that are both in our ETL output file and in the database, and insert just the new ones.

Import database connection settings.

In [ ]:
from db_connection import dbconnection 

In [ ]:
delete_table = """
DROP TABLE IF EXISTS Date;
"""

create_table = """
CREATE TABLE Date (
    DATE_ID INT PRIMARY KEY NOT NULL,
    DAY INT NOT NULL,
    WEEK INT NOT NULL,
    MONTH INT NOT NULL,
    YEAR INT NOT NULL,
    SEASON VARCHAR(10) CHECK(SEASON in ('Spring', 'Summer', 'Autumn', 'Winter')) NOT NULL,
    WEEKEND VARCHAR(11) CHECK (WEEKEND in ('Weekend', 'Work Day')) NOT NULL,
    WEEKDAY VARCHAR(10) CHECK (WEEKDAY IN ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday')) NOT NULL,
    QUARTER VARCHAR(2) CHECK(QUARTER IN ('Q1','Q2','Q3','Q4')) NOT NULL,
    SEMESTER VARCHAR(2) CHECK(SEMESTER IN ('S1','S2')) NOT NULL,
    HOLIDAY VARCHAR(11) CHECK (HOLIDAY in ('Holiday', 'Not Holiday')) NOT NULL
);
"""

In [ ]:
def excuteSingleSQLstatement(sql, host, database, user, password,gssencmode,sslmode):
    conn = pg.connect(host=host,database=database, user=user, password=password,sslmode = sslmode,gssencmode=gssencmode)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

Execute just the first time the notebook runs

In [ ]:
#excuteSingleSQLstatement(delete_table, dbconnection.server_host, dbconnection.dbname, dbconnection.dbusername, dbconnection.dbpassword,gssencmode = dbconnection.gssencmode,sslmode=dbconnection.sslmode)
#excuteSingleSQLstatement(create_table, dbconnection.server_host, dbconnection.dbname, dbconnection.dbusername, dbconnection.dbpassword,gssencmode = dbconnection.gssencmode,sslmode=dbconnection.sslmode)

In [ ]:
# function to query table and convert it to pandas dataframe
def query_table(conn, table_name):
    """Returns DataFrame with queried database table"""
    sql = "select * from {};".format(table_name)
    #return dataframe
    return sqlio.read_sql_query(sql, conn)

# for this function to run, the dataframes must have the same columns, in the same order
def get_data_to_insert(df_etl, df_sql,pk):
    """Returns data valid for insertion in dimension from a new ETL-processed DataFrame"""
    df_etl[pk] = df_etl[pk].astype(int)
    df_sql[pk] = df_sql[pk].astype(int)
    df_insert = df_etl[-df_etl[pk].astype(int).isin(df_sql[pk].astype(int))].dropna(how = 'all')
    df_insert = df_insert.drop_duplicates(subset=[pk])
    return df_insert

# function for bulk insert
def insert_data(df, table_name, conn):
    """Inserts selected data into dimension table in database"""
    df_columns = list(df)
    columns = ",".join(df_columns)
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
    insert_stmt = "INSERT INTO {} ({}) {}".format(table_name,columns,values)
    success = True
    try:
        cursor = conn.cursor()
        psycopg2.extras.execute_batch(cursor, insert_stmt, df.values)
        conn.commit()
        success = True
    except pg.DatabaseError as error:
        success = False
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return success

Query date dimension, from postgres db server

In [ ]:
conn = psycopg2.connect(host = dbconnection.server_host,database = dbconnection.dbname, user = dbconnection.dbusername,password = dbconnection.dbpassword,sslmode=dbconnection.sslmode,gssencmode=dbconnection.gssencmode)
df_date_dimension_sql = query_table(conn, 'date')
conn.close()
df_date_dimension_sql.head()

Guarantee that duplicates will be not be inserted, by getting just the data in dataframe with the ETL output (df_date_dimension_insert) that are not in df_date_dimension_sql.

In [ ]:
df_insert = get_data_to_insert(df_date_dimension_insert,df_date_dimension_sql,'date_id')
df_insert

**Insert data** into database

In [ ]:
if len(df_insert) > 0:
    table_name = 'date'
    conn = psycopg2.connect(host = dbconnection.server_host,database = dbconnection.dbname, user = dbconnection.dbusername,password = dbconnection.dbpassword,sslmode=dbconnection.sslmode,gssencmode=dbconnection.gssencmode)
    success = insert_data(df_insert,table_name, conn)
    conn.close()
    if success == True: print('Data inserted successfully')
else: print('No data to insert')

## Mapping file

Create dataframe that saves the mapping between the listing_id and date_id. This will be used to load the listings fact table.

In [ ]:
df_mapping = df_listings[['id','date_id']].rename(columns = {'id':'listing_id'})
df_mapping.head()

Save csv to file

In [ ]:
df_mapping.to_csv('../processed_dt/df_listings_date.csv')